# What makes a Kickstarter campaign successful?

## Project Overview

Purpose of the project...

Data source...

## Data Cleaning

Information on how the data was cleaned...

In [2]:
# Data Cleaning Code/link to the file

## Exploratory Data Analysis

Purpose of the data exploration...

In [3]:
# Put Visualization #1 code here: 

Analysis of the visualization here...

## Statistical Modeling

In [4]:
# Statistical Modeling code here: 

Explanation of the statistical model and what it means...

## Machine Learning

Before we can do any actual machine learning, we have to first split our data into a test set, and a training set. Since our data set is particularly large, I chose to make the training set a little bit bigger because that will allow the model to have a better fit without worrying about possibility of over fitting. There's still a lot of data in the test set to validate.

In [ ]:
feature = data[['category', 'main_category', 'currency', 'goal','duration', 'country', 'month_launched', 'month_deadline', 'launch_day_of_week', 'count_x']]
outcome = data[['state']]
feature_name = data[["name"]]

We can this pretty easily by first selecting all the relevant features and put those into a data frame. Next, we can put the outcome of the data into it's own data frame as well. We can then split the data into a training set and a test set using the function below.

In [5]:
train_features_small, test_features, train_outcome_small, test_outcome = train_test_split(
    feature,
    outcome,
    test_size = 0.3
)

## Random Forest

We tried many different many different machine learning algorithms over the course of this study but none performed as well as Random Forest. The Random Forest is fairly similar to a Decision Tree algorithm but differs in that a Random Forest builds multiple decision trees in order to get a more accurate prediction.

When tuning the algorithm, we tried various parameters to see which would get us the most accurate model. After a while we found that the best parameters for our model were...

Variance Threshold = .1
SelectKBest = 10
Random Forest Classifier N-estimators = 40

In [1]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_features="auto",max_depth=4, min_samples_split=2)
pipe = make_pipeline(VarianceThreshold(.1),
                     SelectKBest(k=10),
                     rfc
                     )

params ={'randomforestclassifier__n_estimators':[40],
        }

grid_search_rf = GridSearchCV(pipe, params,
                               cv = KFold(n_splits=10, shuffle=True),n_jobs = -1)

C:\Users\Soda\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


NameError: name 'make_pipeline' is not defined

After conducting the GridSearch, we then fit our model to the training data.

In [ ]:
grid_search_rf.fit(train_features_small,train_outcome_small)

We then used this to predict based on our test set.

In [ ]:
grid_search_rf.score(validation_features,validation_outcome)

With an accuracy score of 66%, we can see that our model does a pretty good job at predicting the success or failure of a kickstarter event. But it might be more interesting to see where the model does where it doesn't. 

First we can start by adding a column of our predictions and readding the actual outcome to the test set.

In [ ]:
test_features['preds'] = grid_search_rf.predict(test_features)
test_features['actual'] = test_outcome['state']

We can then calculate measures such as Sensitivity and Specificity. Sensitivity, also referred to as the true positive rate, tells us, of all of the cases in the data, how many did we accurately predict? This indicates the model's ability to detect cases. In other words, how sensitively does the model pick up on cases?

Specificity, also referred to as the true negative rate, tells us, of all of the non-cases in the data, how many did we accurately predict? This indicates the model's ability to assign non-cases.

In [ ]:
true_df = test_features.loc[test_features['actual'] == test_features['preds']][['actual', 'preds']]
false_df = test_features.loc[test_features['actual'] != test_features['preds']][['actual', 'preds']]

true_positives = len(true_df[true_df['preds']==1])
false_negatives = len(false_df[false_df['preds']==0])
true_negatives = len(true_df[true_df['preds']==0])
false_positives = len(false_df[false_df['preds']==1])

sensitivity = true_positives / (true_positives + false_negatives)
specificity = true_negatives / (true_negatives + false_positives)

print(f'Sensitivity: {sensitivity}')
print(f'Specificity: {specificity}')

After calculating these metrics we get a sensitivity value of .136 and a specificity value of .957. This gives is a much clearer understanding of where our model is effective. These values illustrate that our model does not accurately predict positive cases but is really accurate at predicting when a project will fail. This makes a lot of sense because most projects that get created by kickstarter tend to fail so it makes sense that it would be hard to predict whethere a project will succeed just based on the features that we have available to us.

We can visualize this by looking at the ROC Curve. The ROC Curve is a plot of the false positive rate (x-axis) versus the true positive rate (y-axis) for a number of different candidate threshold values between 0.0 and 1.0. Put another way, it plots the false alarm rate versus the hit rate.

In [ ]:
from sklearn import metrics
fpr, tpr, threshold = metrics.roc_curve(y_true=test_features.actual, y_score=test_features.preds)
roc_auc = metrics.auc(fpr, tpr)

# Draw your ROC curve
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

As you can see, our model does predict more true positives than false positives, but the rate is close.

## *About the Team*

Information on the Team...